In [1]:
# Ame Digital - Teste Engenheiro de Dados
#
# Autor: Dennis Cardoso
#
# E-mail: dennis.cardoso@outlook.com
#
# Data: 22 de Dezembro de 2019

In [3]:
#Bibliotecas importação
import re
from datetime import datetime
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as functions

In [4]:
# Iniciar spark context
conf = SparkConf().setMaster('local[*]')
sc = SparkContext().getOrCreate(conf)
sqlc =  SQLContext(sc)

In [5]:
# Declaração do Schema de dados a ser utilizado
schema = StructType([StructField('host',StringType(), True),StructField('timestamp',StringType(), True),StructField('request',StringType(), True),StructField('http_code',IntegerType(), True), StructField('total_bytes',IntegerType(), True)])

In [13]:
#Função que retorna data a partir de um string
def str_date(input_date):
        try:
            return datetime.fromordinal(input_date).strftime('%d/%m/%Y')
        except Exception as e:
            return '01/01/1900'

In [14]:
# Função para fazer parse dos dados de entrada
def parseLog(data):
        ''' Read and parse log data '''
        RE_MASK = '(.*) - - \[(.*):(.*):(.*):(.*)\] "(.*)" ([0-9]*) ([0-9]*|-)'

        try:
            re_result = re.compile(RE_MASK).match(data)
            host = re_result.group(1)
            ord_day = str_date(datetime.strptime(re_result.group(2), '%d/%b/%Y').toordinal())
            req = re_result.group(6)
            reply_code = int(re_result.group(7))
            
            try:
                reply_bytes = int(re_result.group(8))
            except ValueError as e:
                reply_bytes = 0
            return host, ord_day, req, reply_code, reply_bytes
        
        except Exception as e:
            return '', -1, '', -1, -1

In [8]:
# Teste da função Parse_log
valor = '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'
resultado = parseLog(valor)
resultado

('199.72.81.55', '01/07/1995', 'GET /history/apollo/ HTTP/1.0', 200, 6245)

In [17]:
# Gerar RDD com dados dos arquivos
rows = sc.textFile('files')

# Gerar parse dos dados de Log
nasa_parse = rows.map(parseLog)

# remover linhas com valores inválidos
nasa_rdd = nasa_parse.filter(lambda x: x[3] > -1)

In [19]:
# Criação do Dataframe
nasa_df = sqlc.createDataFrame(nasa_rdd, schema)

In [21]:
# Schema do dataframe
nasa_df.printSchema()
nasa_df.select('*').show()

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- request: string (nullable = true)
 |-- http_code: integer (nullable = true)
 |-- total_bytes: integer (nullable = true)

+--------------------+----------+--------------------+---------+-----------+
|                host| timestamp|             request|http_code|total_bytes|
+--------------------+----------+--------------------+---------+-----------+
|        199.72.81.55|01/07/1995|GET /history/apol...|      200|       6245|
|unicomp6.unicomp.net|01/07/1995|GET /shuttle/coun...|      200|       3985|
|      199.120.110.21|01/07/1995|GET /shuttle/miss...|      200|       4085|
|  burger.letters.com|01/07/1995|GET /shuttle/coun...|      304|          0|
|      199.120.110.21|01/07/1995|GET /shuttle/miss...|      200|       4179|
|  burger.letters.com|01/07/1995|GET /images/NASA-...|      304|          0|
|  burger.letters.com|01/07/1995|GET /shuttle/coun...|      200|          0|
|     205.212.115.106|

In [22]:
# Número de HOSTs únicos
hosts_list = nasa_df.groupby('host').agg(functions.count('host').alias('count')).filter("count = 1").count()

In [23]:
# Total De error 404 dentro do Periodo
total_404_errors = nasa_df.filter("http_code = 404")
total_404_errors.count()

20901

In [31]:
# Quais dias do período especificado tiveram o maior número de erros 404.
data_404_rdd = nasa_rdd.map(lambda x: (x[1], 1)) 
data_404_counts = data_404_rdd.reduceByKey(lambda a, b: a+b).sortBy(keyfunc=lambda l: l[1], ascending=False)

In [39]:
data_404_list = data_404_counts.take(5)
for i in data_404_list:
    print('Dia '+i[0]+" : "+str(i[1])+" ocorrências 404")

Dia 13/07/1995 : 134203 ocorrências 404
Dia 06/07/1995 : 100960 ocorrências 404
Dia 05/07/1995 : 94575 ocorrências 404
Dia 12/07/1995 : 92536 ocorrências 404
Dia 31/08/1995 : 90125 ocorrências 404


In [0]:
# O total de bytes retornados no período, com uma visão acumulada
total_404_errors.select('*').show()